# Ejemplo Completo: Regresión Lineal
## Predicción de Precios de Casas

### Objetivo
Predecir el precio de casas basándose en sus características (área, habitaciones, antigüedad).

### Conceptos que aprenderás:
- Cómo crear datos sintéticos para practicar
- Implementar regresión lineal
- Evaluar modelos de regresión
- Interpretar coeficientes
- Hacer predicciones con nuevos datos

---

## 1. Importar Bibliotecas

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración para gráficos
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

---

## 2. Crear Datos Sintéticos

Vamos a generar 200 casas con características aleatorias para simular un dataset real.

In [ ]:
# Fijar semilla para reproducibilidad
np.random.seed(42)
n_samples = 200

# Variables independientes (características)
area = np.random.uniform(50, 300, n_samples)              # metros cuadrados
habitaciones = np.random.randint(1, 6, n_samples)         # número de habitaciones
antiguedad = np.random.uniform(0, 50, n_samples)          # años de antigüedad

# Variable dependiente (precio) con relación matemática + ruido
# Fórmula: precio base depende de área, habitaciones y antigüedad
precio = (area * 1000 +                    # $1000 por m²
          habitaciones * 50000 -            # $50000 por habitación
          antiguedad * 500 +                # -$500 por año de antigüedad
          np.random.normal(0, 20000, n_samples))  # ruido aleatorio

# Crear DataFrame
df = pd.DataFrame({
    'area': area,
    'habitaciones': habitaciones,
    'antiguedad': antiguedad,
    'precio': precio
})

print("=" * 60)
print("DATASET DE CASAS")
print("=" * 60)
print("\nPrimeras 10 filas del dataset:")
print(df.head(10))
print(f"\nForma del dataset: {df.shape}")
print(f"Filas: {df.shape[0]}, Columnas: {df.shape[1]}")

---

## 3. Análisis Exploratorio de Datos (EDA)

In [ ]:
print("\n" + "=" * 60)
print("ESTADÍSTICAS DESCRIPTIVAS")
print("=" * 60)
print(df.describe())

print("\n" + "=" * 60)
print("INFORMACIÓN DEL DATASET")
print("=" * 60)
print(df.info())

print("\n" + "=" * 60)
print("VERIFICAR VALORES FALTANTES")
print("=" * 60)
print(df.isnull().sum())

In [ ]:
# Visualización de las características
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Área vs Precio
axes[0, 0].scatter(df['area'], df['precio'], alpha=0.5, color='blue')
axes[0, 0].set_xlabel('Área (m²)')
axes[0, 0].set_ylabel('Precio ($)')
axes[0, 0].set_title('Relación: Área vs Precio')

# Habitaciones vs Precio
axes[0, 1].scatter(df['habitaciones'], df['precio'], alpha=0.5, color='green')
axes[0, 1].set_xlabel('Número de Habitaciones')
axes[0, 1].set_ylabel('Precio ($)')
axes[0, 1].set_title('Relación: Habitaciones vs Precio')

# Antigüedad vs Precio
axes[1, 0].scatter(df['antiguedad'], df['precio'], alpha=0.5, color='red')
axes[1, 0].set_xlabel('Antigüedad (años)')
axes[1, 0].set_ylabel('Precio ($)')
axes[1, 0].set_title('Relación: Antigüedad vs Precio')

# Distribución de precios
axes[1, 1].hist(df['precio'], bins=30, color='purple', alpha=0.7, edgecolor='black')
axes[1, 1].set_xlabel('Precio ($)')
axes[1, 1].set_ylabel('Frecuencia')
axes[1, 1].set_title('Distribución de Precios')

plt.tight_layout()
plt.show()

In [ ]:
# Matriz de correlación
print("\n" + "=" * 60)
print("MATRIZ DE CORRELACIÓN")
print("=" * 60)
correlation_matrix = df.corr()
print(correlation_matrix)

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Matriz de Correlación entre Variables')
plt.show()

---

## 4. Preparar los Datos

In [ ]:
# Separar características (X) y variable objetivo (y)
X = df[['area', 'habitaciones', 'antiguedad']]
y = df['precio']

print("=" * 60)
print("PREPARACIÓN DE DATOS")
print("=" * 60)
print(f"\nForma de X (características): {X.shape}")
print(f"Forma de y (objetivo): {y.shape}")

# Dividir en conjunto de entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,      # 20% para test
    random_state=42     # Semilla para reproducibilidad
)

print(f"\nDatos de entrenamiento: {X_train.shape[0]} muestras")
print(f"Datos de prueba: {X_test.shape[0]} muestras")
print(f"Proporción: {X_train.shape[0]/len(X)*100:.1f}% train, {X_test.shape[0]/len(X)*100:.1f}% test")

### TIP sobre "Semilla para reproductibilidad" ### 

**"The Hitchhiker’s Guide to the Galaxy"**, novela de ciencia ficción escrita por Douglas Adams

🪐¿Qué tiene que ver con el número 42?  
En la historia, un grupo de seres hiperinteligentes construye una supercomputadora llamada Deep Thought para responder a la pregunta fundamental sobre la vida, el universo y todo lo demás.
Después de 7.5 millones de años de cálculo, la computadora responde:
"La respuesta es... 42."

😲 Pero nadie sabe cuál era la pregunta original, así que la respuesta no tiene sentido sin contexto.  
Es un chiste sobre lo absurdo de buscar una verdad absoluta sin entender bien la pregunta.

🤓 ¿Por qué se usa en programación?
- El número 42 se volvió un símbolo geek y un guiño entre programadores, matemáticos y fans de la ciencia ficción.
- Se usa como semilla aleatoria, valor de prueba o ejemplo genérico, como una forma divertida de decir: "Este número no importa, pero lo elegí con estilo."



---

## 5. Crear y Entrenar el Modelo

In [ ]:
# Crear el modelo de regresión lineal
modelo_regresion = LinearRegression()

print("\n" + "=" * 60)
print("ENTRENAMIENTO DEL MODELO")
print("=" * 60)
print("Entrenando modelo de Regresión Lineal...")

# Entrenar el modelo
modelo_regresion.fit(X_train, y_train)

print("✓ Modelo entrenado exitosamente!")

---

## 6. Hacer Predicciones

In [ ]:
# Hacer predicciones en el conjunto de prueba (40 muestras)
y_pred = modelo_regresion.predict(X_test)

print("\n" + "=" * 60)
print("PREDICCIONES")
print("=" * 60)
print("\nPrimeras 10 predicciones vs valores reales:")
comparacion = pd.DataFrame({
    'Precio Real': y_test.values[:10],
    'Precio Predicho': y_pred[:10],
    'Error': y_test.values[:10] - y_pred[:10]
})
print(comparacion)

---

## 7. Evaluar el Modelo

In [ ]:
# Calcular métricas de evaluación
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\n" + "=" * 60)
print("RESULTADOS DEL MODELO")
print("=" * 60)
print(f"\nError Cuadrático Medio (MSE): ${mse:,.2f}")
print(f"Raíz del Error Cuadrático Medio (RMSE): ${rmse:,.2f}")
print(f"Error Absoluto Medio (MAE): ${mae:,.2f}")
print(f"Coeficiente de Determinación (R²): {r2:.4f}")

print("\n" + "-" * 60)
print("INTERPRETACIÓN:")
print("-" * 60)
print(f"• RMSE: En promedio, las predicciones se desvían ${rmse:,.0f}")
print(f"• MAE: El error absoluto promedio es ${mae:,.0f}")
print(f"• R²: El modelo explica {r2*100:.2f}% de la varianza en los precios")
if r2 > 0.8:
    print("  → Excelente ajuste del modelo")
elif r2 > 0.6:
    print("  → Buen ajuste del modelo")
elif r2 > 0.4:
    print("  → Ajuste moderado del modelo")
else:
    print("  → Ajuste pobre del modelo")

---

## 8. Interpretar los Coeficientes

In [ ]:
print("\n" + "=" * 60)
print("COEFICIENTES DEL MODELO")
print("=" * 60)
print("\nEcuación del modelo:")
print(f"Precio = {modelo_regresion.intercept_:,.2f}")

for feature, coef in zip(X.columns, modelo_regresion.coef_):
    signo = "+" if coef >= 0 else ""
    print(f"         {signo} {coef:,.2f} × {feature}")

print("\n" + "-" * 60)
print("INTERPRETACIÓN DE COEFICIENTES:")
print("-" * 60)
for feature, coef in zip(X.columns, modelo_regresion.coef_):
    if coef > 0:
        print(f"• {feature}: Por cada unidad que aumenta, el precio aumenta ${abs(coef):,.2f}")
    else:
        print(f"• {feature}: Por cada unidad que aumenta, el precio disminuye ${abs(coef):,.2f}")

---

## 9. Visualizar Resultados

In [ ]:
# Gráfico: Valores Reales vs Predicciones
plt.figure(figsize=(12, 5))

# Subplot 1: Scatter plot
plt.subplot(1, 2, 1)
plt.scatter(y_test, y_pred, alpha=0.5, color='blue')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
         'r--', lw=2, label='Predicción perfecta')
plt.xlabel('Precio Real ($)')
plt.ylabel('Precio Predicho ($)')
plt.title('Valores Reales vs Predicciones')
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 2: Distribución de errores
plt.subplot(1, 2, 2)
errores = y_test - y_pred
plt.hist(errores, bins=30, color='green', alpha=0.7, edgecolor='black')
plt.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Error = 0')
plt.xlabel('Error de Predicción ($)')
plt.ylabel('Frecuencia')
plt.title('Distribución de Errores')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Grafico distribucion de valores predichos vs reales
plt.figure(figsize=(10, 6))
plt.hist(y_test, bins=30, alpha=0.5, label='Valores Reales', color='blue', edgecolor='black')
plt.hist(y_pred, bins=30, alpha=0.5, label='Valores Predichos', color='orange', edgecolor='black')
plt.xlabel('Precio ($)')
plt.ylabel('Frecuencia')
plt.title('Distribución de Valores Reales vs Predichos')
plt.legend()
plt.show()


In [ ]:
# Grafico de valores predichos vs reales utilizado kdeplot
plt.figure(figsize=(10, 6))
sns.kdeplot(y_test, label='Valores Reales', color='blue', fill=True, alpha=0.5)
sns.kdeplot(y_pred, label='Valores Predichos', color='orange', fill=True, alpha=0.5)
plt.xlabel('Precio ($)')
plt.ylabel('Densidad')
plt.title('Distribución de Valores Reales vs Predichos (KDE)')
plt.legend()
plt.show()


---

## 10. Hacer Predicciones con Nuevos Datos

In [ ]:
print("\n" + "=" * 60)
print("PREDICCIÓN CON NUEVOS DATOS")
print("=" * 60)

# Crear nuevas casas para predecir
nuevas_casas = np.array([
    [150, 3, 10],   # Casa 1: 150m², 3 habitaciones, 10 años
    [200, 4, 5],    # Casa 2: 200m², 4 habitaciones, 5 años
    [100, 2, 20],   # Casa 3: 100m², 2 habitaciones, 20 años
    [250, 5, 0]     # Casa 4: 250m², 5 habitaciones, nueva
])

# Predecir precios
precios_predichos = modelo_regresion.predict(nuevas_casas)

# Mostrar resultados
print("\nPredicciones para nuevas casas:")
print("-" * 60)
for i, (casa, precio) in enumerate(zip(nuevas_casas, precios_predichos), 1):
    print(f"\nCasa {i}:")
    print(f"  • Área: {casa[0]:.0f} m²")
    print(f"  • Habitaciones: {casa[1]:.0f}")
    print(f"  • Antigüedad: {casa[2]:.0f} años")
    print(f"  → Precio estimado: ${precio:,.2f}")

---

## 11. Conclusiones

### ✓ Lo que aprendimos:
1. **Crear datasets sintéticos** para practicar ML
2. **Implementar regresión lineal** con Scikit-Learn
3. **Evaluar modelos** usando métricas apropiadas (MSE, RMSE, MAE, R²)
4. **Interpretar coeficientes** del modelo
5. **Visualizar resultados** y errores
6. **Hacer predicciones** con datos nuevos

### 📊 Resultados del modelo:
- R² alto indica que el modelo captura bien las relaciones
- RMSE nos dice el error promedio esperado en dólares
- Los coeficientes muestran el impacto de cada característica

### 🎯 Próximos pasos:
- Probar con datos reales (datasets de Kaggle)
- Explorar regresión polinomial para relaciones no lineales
- Aplicar regularización (Ridge, Lasso)
- Feature engineering: crear nuevas características

---

## 📚 Referencias
- Documentación Scikit-Learn: https://scikit-learn.org/stable/modules/linear_model.html
- Dataset de práctica: Boston Housing, California Housing